# Optimization & visualization of the electron density

### import basic stuff

In [1]:
import numpy as np

### import local dftpy module 

In [2]:
from dftpy.field import DirectField
from dftpy.grid import DirectGrid
from dftpy.functional import LocalPseudo, Functional, TotalFunctional
from dftpy.formats import io
from dftpy.math_utils import ecut2nr
from dftpy.time_data import TimeData
from dftpy.optimization import Optimization
from dftpy.mpi import sprint
from dftpy.constants import Units

### simple functions to make a grid and initial guess

In [3]:
def MakeGrid(ions, spacing):
    nr=ecut2nr(lattice=ions.cell, spacing=spacing)
    grid = DirectGrid(lattice=ions.cell, nr=nr, full=False)
    sprint('The final grid size is ', nr)
    return grid

def guess_rho(grid, ions):
    rho_ini = DirectField(grid=grid)
    rho_ini[:] = ions.get_ncharges()/ions.cell.volume
    return rho_ini

### read input, set pseudo files, and make a grid

In [4]:
path_pp=path_pos='../DATA/'
file1='Al_lda.oe01.recpot'
posfile='fcc.vasp'
ions = io.read(path_pos+posfile, names=['Al'])
gap = 0.4
grid = MakeGrid(ions,gap)

The final grid size is  [20 20 20]


### read pseudo file, generate local pseudo, and generate guess density

In [5]:
PP_list = {'Al': path_pp+file1}
PSEUDO = LocalPseudo(grid = grid, ions=ions,PP_list=PP_list,PME=True)

rho_ini = guess_rho(grid,ions)

setting key: Al -> ../DATA/Al_lda.oe01.recpot


### instance KEDF, XC and HARTREE functionals

In [6]:
KE = Functional(type='KEDF',name='TFvW',optional_kwargs={})
XC = Functional(type='XC',name='LDA')
HARTREE = Functional(type='HARTREE')

### instance DFTpy energy evaluator

In [7]:
E_v_Evaluator = TotalFunctional(KE=KE, XC=XC, HARTREE=HARTREE, PSEUDO=PSEUDO)

### instance and execute DFTpy density optimizer

In [8]:
optimization_options = {\
        'econv' : 1e-6*ions.nat, # Energy Convergence
        'maxfun' : 50,  # For TN method, it's the max steps for searching direction
        'maxiter' : 100,# The max steps for optimization
        }
optimization_options["econv"] *= ions.nat
opt = Optimization(EnergyEvaluator=E_v_Evaluator, optimization_options = optimization_options,
        optimization_method = 'TN')


rho = opt.optimize_rho(guess_rho=rho_ini)

Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       2.692153511701E+00      2.692154E+00    7.877088E-01    1       1       1.317573E-02    
1       2.509905183342E+00      -1.822483E-01   7.033259E-02    2       1       2.190518E-02    
2       2.502273083432E+00      -7.632100E-03   4.803592E-03    7       1       4.251766E-02    
3       2.502030159526E+00      -2.429239E-04   3.640522E-04    5       1       5.700088E-02    
4       2.501995243499E+00      -3.491603E-05   3.590655E-05    6       1       7.301950E-02    
5       2.501992596732E+00      -2.646767E-06   2.500795E-06    5       1       8.690786E-02    
6       2.501992226203E+00      -3.705293E-07   4.578793E-08    8       1       1.084733E-01    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.3011519654704218
Chemical potential (eV)  : 8.194761602484691


### evaluate final energy, make a mol system class, and print results

In [9]:
Enew = E_v_Evaluator.Energy(rho=rho, ions=ions)
print('Energy New (a.u.)', Enew)
print('Energy New (eV)', Enew * Units.Ha)
print('Energy New (eV/atom)', Enew * Units.Ha/ions.nat)
print('-' * 31, 'Time information', '-' * 31)
print("{:28s}{:24s}{:20s}".format('Label', 'Cost(s)', 'Number'))
for key in TimeData.cost :
    print("{:28s}{:<24.4f}{:<20d}".format(key, TimeData.cost[key], TimeData.number[key]))
print('-' * 80)

Energy New (a.u.) -8.281138990518853
Energy New (eV) -225.34126979244738
Energy New (eV/atom) -56.335317448111844
------------------------------- Time information -------------------------------
Label                       Cost(s)                 Number              
Optimize                    0.1098                  1                   
TF                          0.0081                  41                  
vW                          0.0200                  41                  
FFT                         0.0121                  83                  
InvFFT                      0.0141                  83                  
LibXC                       0.0474                  41                  
libxc_eval                  0.0286                  82                  
Hartree_Func                0.0156                  41                  
Vion_PME                    0.0026                  1                   
Ewald_Energy                0.0013                  1                   
Ew

### Visualize with ipv

In [10]:
from dftpy.visualize import view

In [11]:
view(data=rho, viewer='ipyvolume')

Container(children=[HBox(children=(FloatSlider(value=0.02019837623202079, max=0.03071556597890547, min=0.01240…

In [12]:
view(ions=ions, viewer='ipyvolume')

Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

### Visualize with VESTA

In [13]:
# view(ions=ions, data=rho, viewer='vesta')